In [ ]:
import requests
import mysql.connector

# API Configuration

API_URL = "https://api.sportradar.com/tennis/trial/v3/en/complexes.json"

API_KEY = "UdfDtENKaaWlf1UzBnzVSldMxYu4umMwr3l1aOLd"

# MySQL Database Configuration (Database: compl)
DB_CONFIG = {
    "host": "localhost",    
    "user": "root",         
    "password": "",         
    "database": "compl"    
}

def fetch_complexes():
    """Fetch complex data from Sportradar API."""
    response = requests.get(API_URL, params={"api_key": API_KEY})
    
    if response.status_code == 200:
        return response.json().get("complexes", [])  # Extract 'complexes' list
    else:
        print("Error fetching data:", response.status_code, response.text)
        return []

def insert_into_db(complexes):
    """Insert complex data into MySQL."""
    try:
        conn = mysql.connector.connect(**DB_CONFIG)
        cursor = conn.cursor()
        
        query = """
        INSERT INTO complexes (complex_id, complex_name) 
        VALUES (%s, %s) 
        ON DUPLICATE KEY UPDATE complex_name = VALUES(complex_name)
        """
        
        data = [(c["id"], c["name"]) for c in complexes]
        
        cursor.executemany(query, data)
        conn.commit()
        
        print(f"{cursor.rowcount} records inserted/updated.")
        
        cursor.close()
        conn.close()
    
    except mysql.connector.Error as err:
        print("Database error:", err)

if __name__ == "__main__":
    complexes = fetch_complexes()
    if complexes:
        insert_into_db(complexes)
    else:
        print("No data to insert.")


632 records inserted/updated.
